# Credit scoring model


### Introduction : Pratique, explication du code et résultats

#### Objectif: créer un algorithme de notation du crédit qui prédit la probabilité qu'un demandeur de prêt soit en défaut de remboursement.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix

# importing necessary libraries

#### Import des du fichiers dans deux dataFrames differents

In [ ]:
df = pd.read_csv("../input/hmeq.csv")
df_i = pd.read_csv("../input/hmeq.csv")
# lecture de l'entrée
# En le stockant dans 2 dataframes, nous effectuerons nos opérations sur df En cas de besoin de table initiale sans aucun changement nous pouvons utiliser df_i

In [ ]:
df.head()

# Aperçu du jeu de donnéesv, les différentes colonnes

# # Comprendre les données

In [ ]:
df.shape

In [ ]:
df.info()

# nombre d'entrées qui ne sont pas c'est à dire  non null 

In [ ]:
df.describe()
# Statistiques descriptives
# Distribution des données
# Il n'y a pas d'anomalies dans les données (respecter les maximums et les moyennes dans chaque cas)

In [ ]:
df.columns
# Colonnes de l'ensemble de données

## Distributions de diverses variables

In [ ]:
print(df["BAD"].value_counts())
df["BAD"].value_counts().plot("barh")
# distribution de la variable cible "BAD"
# La classe cible est un peu déséquilibrée - les zéros sont d'environ 80% et les uns sont d'environ 20%

 distribution de la variable cible "BAD"
# La classe cible est un peu déséquilibrée - les zéros sont d'environ 80% et les uns sont d'environ 20%

In [ ]:
print(df["REASON"].value_counts())

# Ceci est une fonctionnalité nominale, elle doit être modifiée de manière à pouvoir l'utiliser.

In [ ]:
print(df["JOB"].value_counts())

# Identique au cas ci-dessus, nous devons trouver un moyen de l'utiliser.

In [ ]:
df["LOAN"].plot.hist(bins = 20,figsize=(15,7.5))

# distribution de la variable de prêt
# la densité entre 10000-30000 est élevée

In [ ]:
df["DEBTINC"].plot.hist(bins = 20,figsize=(15,5))
 
# Highly populated around 25-50
# We may cap off the end values if required.

#### Distribution de la variable de prêt
#### la densité entre 10000-30000 est élevée

In [ ]:
df["CLAGE"].plot.hist(bins = 20,figsize=(15,7.5))

# La densité est élevée autour de 100-300 # Nous pouvons plafonner les valeurs> = 600 pour obtenir de meilleurs résultats

In [ ]:
df["CLNO"].plot.hist(bins = 20,figsize=(15,5))

# Cette distribution semble bonne et nous n'avons rien à modifier ici.

In [ ]:
df["VALUE"].plot.hist(bins = 80,figsize=(15,7.5))

# La concentration est élevée autour de 80000-100000 # Il y a très moins de valeurs à la fin (> = 400000) qui sont un peu élevées par rapport à la moyenne. Nous pouvons les plafonner.

In [ ]:
df["MORTDUE"].plot.hist(bins = 40,figsize=(15,7.5))

# La concentration est élevée autour de 40000-100000 # Les valeurs à la fin (> = 300000) peuvent être plafonnées.

In [ ]:
df["YOJ"].plot.hist(bins = 40,figsize=(15,7.5))
# C'est très biaisé. Il serait préférable de modifier cette variable pour diminuer l'asymétrie.

In [ ]:
df["DEROG"].value_counts()

# Des incidents dérogatoires n'ont été signalés que dans quelques cas. # Ainsi, créer une variable binaire avec des valeurs 1 pour au moins un incident désobligeant et 0 pour aucun rapport de ce type peut être utile.

In [ ]:
df["DELINQ"].value_counts()

# La plupart d'entre eux sont nuls. # Identique au cas ci-dessus, la création d'une variable binaire serait utile.

In [ ]:
df["NINQ"].value_counts()
# Distribué principalement entre les cinq premières valeurs

### Conclusions: - Les distributions sont correctes et il n'y a pas d'anomalies dans les données. <br> - DEBTINC a un nombre très élevé de données manquantes (sera traité dans la section suivante - Imputation des variables). <br> - La fonction YOJ est fortement biaisée et peut être modifiée pour réduire l'asymétrie. <br> - Caractéristiques nominales: JOB et REASON doivent être modifiés de manière à pouvoir les utiliser pour le modèle de régression logistique. <br> - DELINQ, DEROG peut être divisé en 2 classes pour créer de nouvelles variables binaires. <br> - VALUE, MORTDUE, CLAGE, DEBTINC peuvent être plafonnés à la fin, c'est-à-dire que les valeurs très élevées seront réglées sur une valeur inférieure sélectionnée. -------------------------------------------------- -------------------------------------------------- ---------------------------

# Imputation des variables d'entrée
<br>

In [ ]:
df.isnull().sum()

# Nombre de cas avec Nan.

#### Observations: - Sauf dans le cas de DEBTINC, dans tous les autres cas, seules quelques valeurs n'ont pas été rapportées - Pour imputer les valeurs manquantes, nous pouvons penser à quelques idées comme: - En cas de caractéristiques nominales, les remplacer par la majorité classe - Dans le cas de variables numériques comme DEROG et DELINQ, la plupart des cas sont 0. Nous pouvons les remplacer par la classe majoritaire. - Dans le cas d'autres entrées numériques, nous pouvons les remplacer par la médiane ou la moyenne sans en modifier le plus. Dans ce cahier, je vais les remplacer par la colonne respective.

In [ ]:
df["DEROG"].fillna(value=0,inplace=True)
df["DELINQ"].fillna(value=0,inplace=True)

In [ ]:
# Caractéristiques nominales # Remplacement par classe majoritaire # classe majoritaire en cas de variable JOB est Other # classe majoritaire en cas de REASON varibale est DebtCon

df["REASON"].fillna(value = "DebtCon",inplace = True)
df["JOB"].fillna(value = "Other",inplace = True)

In [ ]:
# Caractéristiques numériques # Remplacement en utilisant la moyenne de chaque classe

df.fillna(value=df.mean(),inplace=True)

In [ ]:
df.isnull().sum()
# Caractéristiques numériques
# Remplacement en utilisant la moyenne de chaque classe

### Regard final sur les données après avoir rempli les valeurs manquantes

In [ ]:
df.head()

# Application des modèles sur les données après imputation
- Application de la classification de base aux données après remplacement / imputation. Vérifions la performance en appliquant à la fois les algorithmes de régression logistique et d'arbre de décision.
- Avant d'appliquer les algorithmes, les données sont divisées en ensembles d'apprentissage et de test dans le rapport 2: 1, soit 33% des données de test et 67% des données de train.
- Et aussi en prenant toutes les colonnes sauf JOB, RAISON comme caractéristiques d'entrée (comme ce sont des caractéristiques nominales, elles doivent être transformées en d'autres variables pour être utilisables, ce qui est pris en compte dans la section suivante).

In [ ]:
# importer les modules requis
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# suppression des fonctionnalités BAD, JOB, REASON de l'ensemble des fonctionnalités d'entrée
x_basic = df.drop(columns=["BAD","JOB","REASON"])
y = df["BAD"]

# Fractionnement des données en ensembles de test et de train
x_basic_tr,x_basic_te,y_tr,y_te = train_test_split(x_basic,y,test_size =.33,random_state=1)
logreg_basic = LogisticRegression()

# Formation du modèle de régression logistique de base avec un ensemble de formation
logreg_basic.fit(x_basic_tr,y_tr)


print("intercept ")
print(logreg_basic.intercept_)
print("")
print("coefficients ")
print(logreg_basic.coef_)

# Prédire la sortie des cas de test à l'aide de l'algorithme créé ci-dessus
y_pre = logreg_basic.predict(x_basic_te)

# Validating the algorithm using various Performance metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print("")
a1 = accuracy_score(y_te,y_pre)
f1 = f1_score(y_te, y_pre, average="macro")
p1 = precision_score(y_te, y_pre, average="macro")
r1 = recall_score(y_te, y_pre, average="macro")
print("accuracy score : ",a1)
print("f1 score : ",f1)
print("precision score : ",p1)
print("recall score : ",r1)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    Cette fonction imprime et trace la matrice de confusion.
    La normalisation peut être appliquée en définissant `normalize = True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Véritable étiquette')
    plt.xlabel('étiquette prédite')

In [ ]:
# Calcul de la matrice de confusion pour l'algorithme ci-dessus

cnf_matrix = confusion_matrix(y_te, y_pre)
np.set_printoptions(precision=2)

# Tracer une matrice de confusion non normalisée
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["BAD"],
                      title='Matrice de confusion - Algorithme de régression logistique')

plt.show()

In [ ]:
# importer les modules requis
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

dectree_basic = DecisionTreeClassifier()
dectree_basic.max_depth = 100
# Formation du modèle d'arbre de décision de base avec un ensemble de formation
dectree_basic.fit(x_basic_tr,y_tr)
# Prédire la sortie des cas de test à l'aide de l'algorithme créé ci-dessus
y_pre = dectree_basic.predict(x_basic_te)
# Validation de l'algorithme à l'aide de diverses métriques de performances

a2 = accuracy_score(y_te,y_pre)
f2 = f1_score(y_te, y_pre, average="macro")
p2 = precision_score(y_te, y_pre, average="macro")
r2 = recall_score(y_te, y_pre, average="macro")
print("accuracy score : ",a2)
print("f1 score : ",f2)
print("precision score : ",p2)
print("recall score : ",r2)

# Calcul de la matrice de confusion pour l'algorithme ci-dessus

cnf_matrix = confusion_matrix(y_te, y_pre)
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["BAD"],
                      title='Confusion matrix,Decision Tree Algorithm')

plt.show()

#### Quelques discussions sur les mesures de performance:
- Généralement, le test de précision peut être utilisé pour évaluer les algorithmes. Mais dans ce cas, le simple fait d'utiliser la CLASSE DE MAJORITÉ (0) pour prédire la sortie donnera une précision élevée (79,2%).
<br>
- Par conséquent, d'autres mesures de performance doivent être utilisées pour évaluer le modèle.
    - Score F1: moyenne pondérée de rappel et de précision
    - Rappel: (TP / TP + FN)
    - Précision: (TP / TP + FP)
      TP est vrai positif, FN est faux négatif, FP est faux positif
<br>
- Ici, nous voulons diminuer le nombre de faux négatifs, c'est-à-dire que nous prévoyons que le crédit sera remboursé mais qu'il s'agit en fait d'un fraudeur. Diminuer FN implique d'augmenter le rappel.Par conséquent, RECALL sera la mesure de performance parfaite pour évaluer ce modèle.
<br>
- La précision peut diminuer dans le processus pour augmenter le rappel, mais il est normal de prédire quelques faux positifs supplémentaires.
##### Nous pouvons également REMPLACER les données (nous y reviendrons à la fin).

#### Conclusions:

- En utilisant la régression logistique bien que la précision soit bonne (79%), le modèle n'a pas bien fonctionné sur les autres mesures de performance.Recal est juste au-dessus de 0,5 et ce n'est pas bon.Cela peut être dû à un surajustement et nous allons essayer de le supprimer dans la section suivante.
<br>
- Étonnamment, l'algorithme de l'arbre de décision a très bien fonctionné par rapport à la régression logistique avec un RAPPEL d'environ 0,78 et une très bonne PRÉCISION, car ce modèle effectue implicitement une sélection de variables / de fonctionnalités en divisant les nœuds supérieurs en fonction des caractéristiques les plus importantes du la sélection des données et des fonctionnalités se fait automatiquement.
<br>
- Enfin ce que je veux dire c'est:
    - Il y aura une bonne amélioration du modèle de régression logistique après la sélection des fonctionnalités.
    - Les résultats resteront presque les mêmes dans le cas du modèle Arbre de décision, même après la sélection des fonctionnalités.
<br>
- Nous allons prouver l'hypothèse ci-dessus en créant des modèles avec des caractéristiques sélectionnées et les comparer aux modèles ci-dessus.

# Transformation des fonctionnalités 

- Avant la sélection des fonctionnalités, comme indiqué dans la section "Distribution des différentes fonctionnalités", nous devons transformer certaines variables afin d'améliorer la prévisibilité.
- Nous avons transformé l'ensemble de données, pas seulement l'ensemble de formation.

In [ ]:
# Couronnement des fonctionnalités CLAGE (valeurs> = 600 à 600), VALUE (valeurs> = 400000 à 400000), MORTDUE (valeurs> = 300000 à 300000) et DEBTINC (valeurs> = 100 à 100)
df.loc[df["CLAGE"]>=600,"CLAGE"] = 600
df.loc[df["VALUE"]>=400000,"VALUE"] = 400000
df.loc[df["MORTDUE"]>=300000,"MORTDUE"] = 300000
df.loc[df["DEBTINC"]>=100,"DEBTINC"] = 100

In [ ]:
# Création de nouvelles variables binaires B_DEROG, B_DELINQ à partir de DEROG, DELINQ

df["B_DEROG"] = (df["DEROG"]>=1)*1
df["B_DELINQ"] = (df["DELINQ"]>=1)*1

In [ ]:
df["JOB"].unique()

In [ ]:
# Nous devons convertir les caractéristiques nominales JOB et RAISON en une forme utilisable et les supprimer de la table de données
df["REASON_1"] = (df["REASON"] == "HomeImp")*1
df["REASON_2"] = (df["REASON"] != "HomeImp")*1
df["JOB_1"] = (df["JOB"]=="Other")*1
df["JOB_2"] = (df["JOB"]=="Office")*1
df["JOB_3"] = (df["JOB"]=="Sales")*1
df["JOB_4"] = (df["JOB"]=="Mgr")*1
df["JOB_5"] = (df["JOB"]=="ProfExe")*1
df["JOB_6"] = (df["JOB"]=="Self")*1
df.drop(["JOB","REASON"],axis = 1,inplace = True)

# L'affectation ci-dessus crée de nouvelles fonctionnalités pour chaque JOB et chaque RAISON

In [ ]:
# Nous devons diminuer l'asymétrie de la fonctionnalité YOU, pour cela nous pouvons appliquer le log de YOU mais comme certains d'entre eux sont 0, nous utiliserons log (constante YOJ)
df["YOJ"] = df["YOJ"].apply(lambda t : np.log(t+1))

In [ ]:
df.head()

* # Sélection de fonctionnalité

- Au fur et à mesure que nous avons terminé la partie transformation, nous passons maintenant à la sélection des fonctionnalités. Nous allons maintenant découvrir les fonctionnalités d'importation les plus affectant la variable cible "MAUVAIS".
- Nous utiliserons les éléments suivants à cette fin:
    - Facteur de corrélation de Pearson Pearson
    - test du chi carré
    - f_régression
    - f_classif

### Utilisation du facteur de corrélation de Pearson pour la sélection des fonctionnalités

In [ ]:
# Recherche de corrélation entre toutes les fonctionnalités et la fonctionnalité cible "BAD"

df.corr(method='pearson')

In [ ]:
# Rassembler les 2 ensembles de fonctionnalités avec une valeur de corrélation élevée, une avec 7 et l'autre avec 10 fonctionnalités

feat1=["DEROG","DELINQ","CLAGE","NINQ","DEBTINC","YOJ","LOAN"]
#feat2=["DEROG","DELINQ","CLAGE","NINQ","DEBTINC","LOAN","JOB_2","YOJ","JOB_3","MORTDUE"]

## Maintenant que nous avons les fonctionnalités avec une forte corrélation avec la fonctionnalité BAD, nous allons exécuter les algorithmes de classification et les comparer

In [ ]:
# Régression logistique à l'aide de l'ensemble de fonctionnalités 1 ci-dessus

x = df[feat1]
y = df["BAD"]
x_tr,x_te,y_tr,y_te = train_test_split(x,y,test_size = 0.33,random_state=1)
logreg = LogisticRegression()
logreg.fit(x_tr,y_tr)
y_pre = logreg.predict(x_te)
a3 = accuracy_score(y_te,y_pre)
f3 = f1_score(y_te, y_pre, average="macro")
p3 = precision_score(y_te, y_pre, average="macro")
r3 = recall_score(y_te, y_pre, average="macro")
print("accuracy score : ",a3)
print("f1 score : ",f3)
print("precision score : ",p3)
print("recall score : ",r3)

# Calcul de la matrice de confusion pour l'algorithme ci-dessus

cnf_matrix = confusion_matrix(y_te, y_pre)
np.set_printoptions(precision=2)

# Tracer une matrice de confusion non normalisée
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["BAD"],
title='Matrice de confusion - Algorithme de régression logistique avec Pearson corr_f')

plt.show()

In [ ]:
# Classificateur d'arbre de décision utilisant feat1

clf_tree=DecisionTreeClassifier()
clf_tree.max_depth = 100
clf_tree.fit(x_tr,y_tr)
y_pre = clf_tree.predict(x_te)
a4 = accuracy_score(y_te,y_pre)
f4 = f1_score(y_te, y_pre, average="macro")
p4 = precision_score(y_te, y_pre, average="macro")
r4 = recall_score(y_te, y_pre, average="macro")
print("accuracy score : ",a4)
print("f1 score : ",f4)
print("precision score : ",p4)
print("recall score : ",r4)
print("")
# Calcul de la matrice de confusion pour l'algorithme ci-dessus

cnf_matrix = confusion_matrix(y_te, y_pre)
np.set_printoptions(precision=2)

# Tracer une matrice de confusion non normalisée
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["BAD"],
title='Matrice de confusion - Algorithme d\'arbre de décision utilisant Pearson corr_f')

plt.show()

### Utilisation du test chi2 pour la sélection des fonctionnalités

In [ ]:
# Trouver les 10 meilleures fonctionnalités à l'aide du test chi2

from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
df_new = pd.DataFrame(SelectKBest(chi2, k=10).fit_transform(df.drop(["BAD"],axis = 1),df["BAD"]))

In [ ]:
# dataframe contenant les fonctionnalités sélectionnées

df_new.head()

In [ ]:
# Exécution de l'algorithme de régression logistique en utilisant les fonctionnalités sélectionnées à partir du test chi2
x = df_new
y = df["BAD"]
x_tr,x_te,y_tr,y_te = train_test_split(x,y,test_size = .33,random_state=1)
logreg = LogisticRegression()
logreg.fit(x_tr,y_tr)
y_pre = logreg.predict(x_te)
y_pre = logreg.predict(x_te)
a5 = accuracy_score(y_te,y_pre)
f5 = f1_score(y_te, y_pre, average="macro")
p5 = precision_score(y_te, y_pre, average="macro")
r5 = recall_score(y_te, y_pre, average="macro")
print("accuracy score : ",a5)
print("f1 score : ",f5)
print("precision score : ",p5)
print("recall score : ",r5)

# Calcul de la matrice de confusion pour l'algorithme ci-dessus

cnf_matrix = confusion_matrix(y_te, y_pre)
np.set_printoptions(precision=2)

# Tracer une matrice de confusion non normalisée
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["BAD"],
 title='Matrice de confusion - Algorithme de régression logistique avec test chi2')

plt.show()


In [ ]:
# Classificateur d'arbre de décision utilisant les fonctionnalités du test chi2

clf_tree=DecisionTreeClassifier()
clf_tree.max_depth = 100
clf_tree.fit(x_tr,y_tr)
y_pre = clf_tree.predict(x_te)
a6 = accuracy_score(y_te,y_pre)
f6 = f1_score(y_te, y_pre, average="macro")
p6 = precision_score(y_te, y_pre, average="macro")
r6 = recall_score(y_te, y_pre, average="macro")
print("accuracy score : ",a6)
print("f1 score : ",f6)
print("precision score : ",p6)
print("recall score : ",r6)
# Calcul de la matrice de confusion pour l'algorithme ci-dessus

cnf_matrix = confusion_matrix(y_te, y_pre)
np.set_printoptions(precision=2)

# Calcul de la matrice de confusion pour l'algorithme ci-dessus
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["BAD"],
title='Matrice de confusion - Algorithme d\'arbre de décision utilisant le test chi2 pour la sélection des fonctionnalités')

plt.show()

In [ ]:
df.head()

## Utilisation de f_classif pour la sélection des fonctionnalités

In [ ]:
from sklearn.feature_selection import f_classif

df_new2 = pd.DataFrame(SelectKBest(f_classif, k=10).fit_transform(df.drop(["BAD"],axis=1),df["BAD"]))
df_new2.head()

In [ ]:
# Exécution de l'algorithme de régression logistique en utilisant les fonctionnalités sélectionnées dans le test f_classif
x = df_new2
y = df["BAD"]
x_tr,x_te,y_tr,y_te = train_test_split(x,y,test_size = .33,random_state=1)
logreg = LogisticRegression()
logreg.fit(x_tr,y_tr)
y_pre = logreg.predict(x_te)
a7 = accuracy_score(y_te,y_pre)
f7 = f1_score(y_te, y_pre, average="macro")
p7 = precision_score(y_te, y_pre, average="macro")
r7 = recall_score(y_te, y_pre, average="macro")
print("accuracy score : ",a7)
print("f1 score : ",f7)
print("precision score : ",p7)
print("recall score : ",r7)

# Calcul de la matrice de confusion pour l'algorithme ci-dessus

cnf_matrix = confusion_matrix(y_te, y_pre)
np.set_printoptions(precision=2)

# Tracer une matrice de confusion non normalisée
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["BAD"],
title='Confusion matrix - Logistic Regression Algorithm with f_classif')

plt.show()

In [ ]:
# Decision Tree classifier using features from f_classif test

clf_tree=DecisionTreeClassifier()
clf_tree.max_depth = 100
clf_tree.fit(x_tr,y_tr)
y_pre = clf_tree.predict(x_te)
a8 = accuracy_score(y_te,y_pre)
f8 = f1_score(y_te, y_pre, average="macro")
p8 = precision_score(y_te, y_pre, average="macro")
r8 = recall_score(y_te, y_pre, average="macro")
print("accuracy score : ",a8)
print("f1 score : ",f8)
print("precision score : ",p8)
print("recall score : ",r8)
# Calcul de la matrice de confusion pour l'algorithme ci-dessus

cnf_matrix = confusion_matrix(y_te, y_pre)
np.set_printoptions(precision=2)

# Tracer une matrice de confusion non normalisée
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["BAD"],
                      title='Confusion matrix - Decision Tree Algorithm using f_classif feature selector')

plt.show()

 # Pour visualiser l'arbre de décision créé -

In [ ]:
from sklearn import tree
import graphviz 
dot_dat = tree.export_graphviz(clf_tree, out_file=None) 
graph = graphviz.Source(dot_dat) 
graph

Utilisation de f_regression pour la sélection des fonctionnalités

In [ ]:
from sklearn.feature_selection import f_regression

df_new3 = pd.DataFrame(SelectKBest(f_regression, k=10).fit_transform(df.drop(["BAD"],axis=1),df["BAD"]))
df_new3.head()

In [ ]:
# Exécution de l'algorithme de régression logistique en utilisant les fonctionnalités sélectionnées dans le test de f_regression

x = df_new3
y = df["BAD"]
x_tr,x_te,y_tr,y_te = train_test_split(x,y,test_size = .33,random_state=1)
logreg = LogisticRegression()
logreg.fit(x_tr,y_tr)
y_pre2 = logreg.predict(x_te)
a9 = accuracy_score(y_te,y_pre2)
f9 = f1_score(y_te, y_pre2, average="macro")
p9 = precision_score(y_te, y_pre2, average="macro")
r9 = recall_score(y_te, y_pre2, average="macro")
print("accuracy score : ",a9)
print("f1 score : ",f9)
print("precision score : ",p9)
print("recall score : ",r9)

# Calcul de la matrice de confusion pour l'algorithme ci-dessus

cnf_matrix = confusion_matrix(y_te, y_pre)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["BAD"],
                      title='Matrice de confusion - Algorithme de régression logistique avec f_regression')

plt.show()

In [ ]:
# Classificateur d'arbre de décision utilisant les fonctionnalités du test f_regression

clf_tree=DecisionTreeClassifier()
clf_tree.max_depth = 100
clf_tree.fit(x_tr,y_tr)
y_pre = clf_tree.predict(x_te)
a10 = accuracy_score(y_te,y_pre)
f10 = f1_score(y_te, y_pre, average="macro")
p10= precision_score(y_te, y_pre, average="macro")
r10 = recall_score(y_te, y_pre, average="macro")
print("accuracy score : ",a10)
print("f1 score : ",f10)
print("precision score : ",p10)
print("recall score : ",r10)

# Computing Confusion matrix for the above algorithm

cnf_matrix = confusion_matrix(y_te, y_pre)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["BAD"], 
                      title='Confusion matrix - Decision Tree Algorithm using f_regression feature selector')

plt.show()

# Comparaison de tous les modèles
 - Nous pouvons maintenant classer notre évaluation de tous les modèles pour choisir le meilleur pour notre problème.

In [ ]:
models = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree','Logistic Regression', 'Decision Tree','Logistic Regression', 'Decision Tree','Logistic Regression', 'Decision Tree','Logistic Regression', 'Decision Tree'],
    'Feature Selection Method' : ['None','None','Pearson corr_fact','Pearson corr_fact','chi2 test','chi2 test','f_classif','f_classif','f_regression','f_regression'],
    'Accuracy Score': [a1,a2,a3,a4,a5,a6,a7,a8,a9,a10],
    'Recall Score' : [r1,r2,r3,r4,r5,r6,r7,r8,r9,r10],
    'F1 Score' : [f1,f2,f3,f4,f5,f6,f7,f8,f9,f10],
    'Precision Score' : [p1,p2,p3,p4,p5,p6,p7,p8,p9,p10]
})

In [ ]:
models

In [ ]:
pd.pivot_table(models,index = ["Feature Selection Method","Model"])

# Discussion et idées:
- La régression logistique a produit des résultats avec une bonne précision mais les performances globales ne sont pas très bonnes.
<br>
- L'arbre de décision a dominé la régression logistique dans tous les cas.
<br>
- Comme mentionné précédemment, les performances de l'arbre de décision sont restées presque les mêmes depuis le début car il sélectionne les fonctionnalités de manière héréditaire.Les performances de la régression logistique se sont également améliorées après le processus de sélection des fonctionnalités.
<br>
- Enfin, le modèle d'arbre de décision avec le sélecteur de fonctionnalités f_classf serait la meilleure méthode à utiliser car il a la valeur RECALL la plus élevée
<br>
- La profondeur maximale de l'arbre de décision est fixée à 100 dans tous les cas, donc le nombre de niveaux est de 101 dans tous les cas. Et puisque nous n'avons pas fixé le nombre minimum d'observations dans la feuille sera de 1 car il s'agit d'un problème de classification.
<br>
- Le seuil est défini par défaut sur 0,5 dans la régression logistique!
<br>
- Évidemment, la modification du seuil affecte les performances du modèle et cela peut être observé dans la section suivante.
<br>
- Cela peut être encore étendu en rééchantillonnant les données pour augmenter le score RECALL

#### Using decision tree with f_classif feature selector would give the best results!!

# Modification du seuil et observation des performances:

In [ ]:
lr = LogisticRegression()
lr.fit(x_tr,y_tr)
y_pred_proba = lr.predict_proba(x_te)

thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

plt.figure(figsize=(10,10))

j = 1
for i in thresholds:
    y_test_predictions_high_recall = y_pred_proba[:,1] >= i
    
    plt.subplot(3,3,j)
    j += 1
    
    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_te,y_test_predictions_high_recall)
    np.set_printoptions(precision=2)
    rec1 = recall_score(y_te, y_test_predictions_high_recall)
    acc= 1.0*(cnf_matrix[0,0]+cnf_matrix[1,1])/(cnf_matrix[0,0]+cnf_matrix[1,0]+cnf_matrix[1,1]+cnf_matrix[0,1])
    print("Recall metric in the testing dataset: ",rec1)
    print("Accuracy score for the testing dataset: ",acc)
    # Plot non-normalized confusion matrix
    class_names = [0,1]
    plot_confusion_matrix(cnf_matrix
                          , classes=class_names
                          , title='Threshold >= %s'%i)
    print("")

#### Conclusion :
- Les valeurs de rappel et de précision varient en fonction du seuil sélectionné.
- Sur la base de la précision requise et des valeurs de rappel, il faut décider et sélectionner un seuil.
- Il est suggéré de passer au seuil par défaut qui est de 0,5 dans les cas généraux.
<br>
-------------------------------------------------- -------------------------------------------------- --------------------------

# Plus à ce sujet: Utilisation du RESAMPLING pour augmenter la valeur de rappel

- Comme mentionné précédemment, nous pouvons utiliser le rééchantillonnage pour améliorer les performances des algorithmes d'apprentissage.
- Dans cette méthode, nous allons diviser les données pour obtenir le rapport de classe cible 1: 1.
- Il s'agit essentiellement d'une méthode qui traitera les données pour avoir un rapport d'environ 50-50.
- Il existe 2 processus pour ce faire, sous-échantillonnage et sur-échantillonnage. Ici, nous allons utiliser UNDER-SAMPLING.

In [ ]:
# obtenir la longueur et les indices de la classe minoritaire.
default_len = len(df[df["BAD"]==1])
default_indices = np.array(df[df["BAD"]==1].index)

# sélectionner le même nombre d'éléments de la classe majoritaire au hasard.
good_indices = np.array(df[df["BAD"]==0].index)
rand_good_indices = np.random.choice(good_indices, default_len, replace = False)
rand_good_indices = np.array(rand_good_indices)

# combing the indices
combined_indices = np.concatenate([rand_good_indices,default_indices])

# getting the corresponding dataset with above indices.
comb_df = df.iloc[combined_indices,:]
comb_y = comb_df["BAD"]

In [ ]:
# en utilisant la méthode de sélection de fonctionnalités f_classif qui a produit de bons résultats dans les cas ci-dessus

from sklearn.feature_selection import f_classif

comb_x = pd.DataFrame(SelectKBest(f_classif, k=10).fit_transform(comb_df.drop(["BAD"],axis=1),comb_df["BAD"]))
comb_x.head()

In [ ]:
# fractionnement des données en ensembles de données de train et de test

x_trc,x_tec,y_trc,y_tec = train_test_split(comb_x,comb_y,test_size =.33,random_state=1000)

In [ ]:
# utiliser les scores Kfold pour entraîner les données car très peu de données sont disponibles

from sklearn.cross_validation import KFold, cross_val_score

lr = LogisticRegression()

def printing_Kfold_scores(x_trc,y_trc):
    fold = KFold(len(y_trc),4,shuffle=False) 
    for train,test in fold :  
        x1 = x_trc.iloc[train,:]
        y1 = y_trc.iloc[train]
        x2 = x_trc.iloc[test,:]
        y2 = y_trc.iloc[test]
        lr.fit(x1,y1)
        y_pred_undersample = lr.predict(x2)
        recall_acc = recall_score(y2,y_pred_undersample)
        print(recall_acc)  
        
printing_Kfold_scores(x_trc,y_trc)

y_predr = lr.predict(x_tec)

print("")
print('Accuracy Score = ',accuracy_score(y_tec,y_predr))
print('F1 Score = ',f1_score(y_tec, y_predr, average="macro"))
print('Precision Score = ',precision_score(y_tec, y_predr, average="macro"))
print('Recall Score = ',recall_score(y_tec, y_predr, average="macro"))
print("")
cnf_matrix = confusion_matrix(y_tec, y_predr)
np.set_printoptions(precision=2)

# Tracer une matrice de confusion non normalisée
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["BAD"], 
                      title='Confusion matrix - Logistic Regression Algorithm after Resampling the data')

plt.show()

In [ ]:
lr = DecisionTreeClassifier()

def printing_Kfold_scores(x_trc,y_trc):
    
    print("# Tracer une matrice de confusion non normalisée")
    fold = KFold(len(y_trc),4,shuffle=False) 
    for train,test in fold :  
        x1 = x_trc.iloc[train,:]
        y1 = y_trc.iloc[train]
        x2 = x_trc.iloc[test,:]
        y2 = y_trc.iloc[test]
        lr.fit(x1,y1)
        y_pred_undersample = lr.predict(x2)
        recall_acc = recall_score(y2,y_pred_undersample)
        print(recall_acc)
        
printing_Kfold_scores(x_trc,y_trc)

y_predr = lr.predict(x_tec)
print("")
print('Accuracy Score = ',accuracy_score(y_tec,y_predr))
print('F1 Score = ',f1_score(y_tec, y_predr, average="macro"))
print('Precision Score = ',precision_score(y_tec, y_predr, average="macro"))
print('Recall Score = ',recall_score(y_tec, y_predr, average="macro"))
print("")

cnf_matrix = confusion_matrix(y_tec, y_predr)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["BAD"], 
                      title='Confusion matrix - Decision Tree Algorithm after Resampling the data')

plt.show()

#### Conclusion :
- Comme vous pouvez le voir, Recall a AUGMENTÉ d'environ 5 à 7% en cas de régression logistique en rééchantillonnant les données. C'est une énorme réussite !!
<br>
- En cas d'algorithme d'arbre de décision, le rappel est plus ou moins le même.